In [4]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-01-20 20:28:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-01-20 20:28:05 (8.53 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("KitchenAmazonReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

kitchen_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
kitchen_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [8]:
# Count the number of the reviews 
print(kitchen_df.count())

4880466


In [9]:
# Drop duplicates and incomplete rows 
dropped_kitchen_df = kitchen_df.dropna()
print(dropped_kitchen_df.count())

4879961


In [10]:
# Drop duplicates and incomplete rows
clean_kitchen_df = dropped_kitchen_df.dropDuplicates()
print(clean_kitchen_df.count())

4879961


In [11]:
# Check the schema
clean_kitchen_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
# Create a new DataFrame for review info
review_id_table_kitchen = clean_kitchen_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_kitchen.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002B10PD9ZFM|   24197845|B000LTOCSG|     630072149| 2009-07-20|
|R1007DQP1BG5WJ|   17990743|B007M2OHEY|     362955205| 2014-09-12|
|R100AIJ0ZFN8CZ|   33816354|B0001ZYYQU|     331822491| 2007-03-09|
|R100C48B994FWD|   13231777|B00IK59T1S|     744019201| 2014-11-26|
|R100E300CKVFYD|   53006884|B00833DQCU|     298219798| 2013-12-08|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [13]:
# Create a new DataFrame for unique values
products_kitchen = clean_kitchen_df.select(["product_id", "product_title"])
products_kitchen.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000LTOCSG|Replacement Filte...|
|B007M2OHEY|Fred & Friends E....|
|B0001ZYYQU|KitchenAid Artisa...|
|B00IK59T1S|PUR 7 Cup Pitcher...|
|B00833DQCU|Prepworks from Pr...|
+----------+--------------------+
only showing top 5 rows



In [14]:
# Total customers count 
customer_count = clean_kitchen_df.select(["customer_id"]).count()
customer_count

4879961

In [15]:
# Create a new DataFrame for customers
customers_kitchen = clean_kitchen_df.groupBy('customer_id').count()
customers_kitchen_counts = customers_kitchen.withColumnRenamed('count', 'customers_count')
customers_kitchen_counts.show(5)

+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   16308995|              2|
|    4709446|              1|
|   31546122|              2|
|   25866981|              7|
|   11537699|              4|
+-----------+---------------+
only showing top 5 rows



In [16]:
# Vine DataFrame 
vine_kitchen = clean_kitchen_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_kitchen.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1002B10PD9ZFM|          5|            0|          0|   N|
|R1007DQP1BG5WJ|          5|            0|          0|   N|
|R100AIJ0ZFN8CZ|          4|            3|          3|   N|
|R100C48B994FWD|          5|            0|          0|   N|
|R100E300CKVFYD|          1|            3|          3|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [1]:
# Write DataFrame to RDS
# Configuration for RDS instance

mode="append"
jdbc_url = "jdbc:postgresql://<input endpoint>:5432/<database name>"
config = {"user":"<input username>",
          "password": "<input password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

review_id_table_kitchen.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

products_kitchen.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

customers_kitchen_counts.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
# Write DataFrame to table

vine_kitchen.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)